### Objective: Create POI vector attribute
Steps:
1. Combine POI information, namely mrt stations, primary schools and malls.
2. Convert the POI text information into vectors for training dataset.

In [18]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import spacy

#### 1.1 Combine POI info

In [85]:
hdb_working_data = pd.read_csv('../data/modified/hdb_working_data.csv')
hdb_working_data.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,longitude,sora,mrt_stations_within_1km,nearest_mrt_station,bto_within_4km,bto_supply_within_4km,pri_schs_within_1km,count_pri_schs_within_1km,distance2cbd,mallsWithin2kmRadius
0,0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,...,103.837619,0.129019,[],Yio Chu Kang MRT Station (1099.56m),12,6587,"['Ang Mo Kio Primary School (676.95m)', ""CHIJ ...",3,9.764087,"['AMK Hub (1365.57m)', 'Broadway Plaza (985.61..."
1,1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,...,103.855621,0.129019,['Ang Mo Kio MRT Station (811.53m)'],Ang Mo Kio MRT Station (811.53m),39,23252,['Jing Shan Primary School (431.03m)'],1,9.515643,"['AMK Hub (937.75m)', 'Broadway Plaza (1103.92..."
2,2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,...,103.838169,0.129019,[],Yio Chu Kang MRT Station (1183.8m),10,4941,"['Ang Mo Kio Primary School (495.36m)', ""CHIJ ...",3,9.585589,"['AMK Hub (1238.44m)', 'Broadway Plaza (878.7m..."
3,3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,...,103.855357,0.129019,['Ang Mo Kio MRT Station (703.32m)'],Ang Mo Kio MRT Station (703.32m),34,20043,"['Jing Shan Primary School (611.18m)', 'Teck G...",3,8.833708,"['AMK Hub (784.06m)', 'Broadway Plaza (1149.31..."
4,4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,103.857736,0.129019,['Ang Mo Kio MRT Station (939.42m)'],Ang Mo Kio MRT Station (939.42m),45,26356,['Jing Shan Primary School (627.43m)'],1,9.275781,"['AMK Hub (1057.2m)', 'Broadway Plaza (1317.66..."


In [86]:
import ast 

hdb_working_data['mrt_stations_within_1km'] = hdb_working_data.apply(lambda row: ' '.join(ast.literal_eval(row['mrt_stations_within_1km'])), axis=1)
hdb_working_data['pri_schs_within_1km'] = hdb_working_data.apply(lambda row: ' '.join(ast.literal_eval(row['pri_schs_within_1km'])), axis=1)
hdb_working_data['mallsWithin2kmRadius'] = hdb_working_data.apply(lambda row: ' '.join(ast.literal_eval(row['mallsWithin2kmRadius'])), axis=1)

In [88]:
hdb_working_data.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,longitude,sora,mrt_stations_within_1km,nearest_mrt_station,bto_within_4km,bto_supply_within_4km,pri_schs_within_1km,count_pri_schs_within_1km,distance2cbd,mallsWithin2kmRadius
0,0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,...,103.837619,0.129019,,Yio Chu Kang MRT Station (1099.56m),12,6587,Ang Mo Kio Primary School (676.95m) CHIJ St. N...,3,9.764087,AMK Hub (1365.57m) Broadway Plaza (985.61m) Ju...
1,1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,...,103.855621,0.129019,Ang Mo Kio MRT Station (811.53m),Ang Mo Kio MRT Station (811.53m),39,23252,Jing Shan Primary School (431.03m),1,9.515643,AMK Hub (937.75m) Broadway Plaza (1103.92m) Ju...
2,2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,...,103.838169,0.129019,,Yio Chu Kang MRT Station (1183.8m),10,4941,Ang Mo Kio Primary School (495.36m) CHIJ St. N...,3,9.585589,AMK Hub (1238.44m) Broadway Plaza (878.7m) Jub...
3,3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,...,103.855357,0.129019,Ang Mo Kio MRT Station (703.32m),Ang Mo Kio MRT Station (703.32m),34,20043,Jing Shan Primary School (611.18m) Teck Ghee P...,3,8.833708,AMK Hub (784.06m) Broadway Plaza (1149.31m) Ju...
4,4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,103.857736,0.129019,Ang Mo Kio MRT Station (939.42m),Ang Mo Kio MRT Station (939.42m),45,26356,Jing Shan Primary School (627.43m),1,9.275781,AMK Hub (1057.2m) Broadway Plaza (1317.66m) Ju...


In [89]:
hdb_working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211395 entries, 0 to 211394
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 211395 non-null  int64  
 1   month                      211395 non-null  object 
 2   town                       211395 non-null  object 
 3   flat_type                  211395 non-null  object 
 4   block                      211395 non-null  object 
 5   street_name                211395 non-null  object 
 6   storey_range               211395 non-null  object 
 7   floor_area_sqm             211395 non-null  float64
 8   flat_model                 211395 non-null  object 
 9   lease_commence_date        211395 non-null  int64  
 10  remaining_lease            197089 non-null  float64
 11  resale_price               211395 non-null  float64
 12  address                    211395 non-null  object 
 13  latitude                   21

In [90]:
hdb_working_data['poi_mrt_prisch_mall'] = hdb_working_data.apply(
    lambda row: ' '.join([
        str(row['mrt_stations_within_1km']),
        str(row['pri_schs_within_1km']),
        str(row['mallsWithin2kmRadius'])
    ]), axis=1)
hdb_working_data.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,sora,mrt_stations_within_1km,nearest_mrt_station,bto_within_4km,bto_supply_within_4km,pri_schs_within_1km,count_pri_schs_within_1km,distance2cbd,mallsWithin2kmRadius,poi_mrt_prisch_mall
0,0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,...,0.129019,,Yio Chu Kang MRT Station (1099.56m),12,6587,Ang Mo Kio Primary School (676.95m) CHIJ St. N...,3,9.764087,AMK Hub (1365.57m) Broadway Plaza (985.61m) Ju...,Ang Mo Kio Primary School (676.95m) CHIJ St. ...
1,1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,...,0.129019,Ang Mo Kio MRT Station (811.53m),Ang Mo Kio MRT Station (811.53m),39,23252,Jing Shan Primary School (431.03m),1,9.515643,AMK Hub (937.75m) Broadway Plaza (1103.92m) Ju...,Ang Mo Kio MRT Station (811.53m) Jing Shan Pri...
2,2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,...,0.129019,,Yio Chu Kang MRT Station (1183.8m),10,4941,Ang Mo Kio Primary School (495.36m) CHIJ St. N...,3,9.585589,AMK Hub (1238.44m) Broadway Plaza (878.7m) Jub...,Ang Mo Kio Primary School (495.36m) CHIJ St. ...
3,3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,...,0.129019,Ang Mo Kio MRT Station (703.32m),Ang Mo Kio MRT Station (703.32m),34,20043,Jing Shan Primary School (611.18m) Teck Ghee P...,3,8.833708,AMK Hub (784.06m) Broadway Plaza (1149.31m) Ju...,Ang Mo Kio MRT Station (703.32m) Jing Shan Pri...
4,4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,0.129019,Ang Mo Kio MRT Station (939.42m),Ang Mo Kio MRT Station (939.42m),45,26356,Jing Shan Primary School (627.43m),1,9.275781,AMK Hub (1057.2m) Broadway Plaza (1317.66m) Ju...,Ang Mo Kio MRT Station (939.42m) Jing Shan Pri...


In [91]:
hdb_working_data['poi_mrt_prisch_mall'].unique()

array([" Ang Mo Kio Primary School (676.95m) CHIJ St. Nicholas Girls' School (415.34m) Mayflower Primary School (232.85m) AMK Hub (1365.57m) Broadway Plaza (985.61m) Jubilee Square (1190.72m)",
       'Ang Mo Kio MRT Station (811.53m) Jing Shan Primary School (431.03m) AMK Hub (937.75m) Broadway Plaza (1103.92m) Jubilee Square (901.91m) myVillage At Serangoon Garden (1443.87m)',
       " Ang Mo Kio Primary School (495.36m) CHIJ St. Nicholas Girls' School (435.97m) Mayflower Primary School (389.34m) AMK Hub (1238.44m) Broadway Plaza (878.7m) Jubilee Square (1089.46m)",
       ...,
       "Hougang MRT Station (589.72m) CHIJ Our Lady of the Nativity (713.85m) Holy Innocents' Primary School (156.46m) Hougang Mall (614.17m) Heartland Mall (1424.46m) Hougang 1 (1328.15m) Hougang Green Shopping Mall (1583.74m) Hougang Rivercourt (559.02m)",
       'Boon Keng MRT Station (272.42m) Bendemeer MRT Station (891.98m) Geylang Bahru MRT Station (965.74m) Bendemeer Primary School (299.17m) Hong Wen Sc

In [92]:
hdb_working_data.to_csv('../data/modified/hdb_working_data_v2.csv', index=False)

#### 2.1 Word embedding
Word embedding is done using a pre-training medium-size model training on web text, designed to support a wide range of natural language processing tasks such as part-of-speech tagging, named entity recognition, dependency parsing, taken from spaCy library.

Each word in the POI text information is first vectorised, and then all vectors aggregated to represent the density vector.

In [9]:
nlp = spacy.load("en_core_web_md")

In [96]:
def embed_poi_density(text):
    print('.', end='', flush=True)

    doc = nlp(text)

    # Initialize an empty array to store word vectors
    word_vecs = []

    # Iterate through each word and append its vector representation
    for token in doc:
        if token.has_vector:
            word_vecs.append(token.vector)

    # Aggregate word vectors to represent the density of POIs
    if len(word_vecs) > 0:
        density_vector = np.mean(word_vecs, axis=0)
    else:
        # If no valid word vectors were found, create a zero vector
        density_vector = np.zeros(nlp.vocab.vectors_length)

    return density_vector

In [ ]:
text = "100 AM (1300m) Dhoby Ghaut MRT station (10m)"
text1 = "Jurong East MRT Station (200m) Bukit Batok MRT Station (800m) AMK Hub (8128.9m) Broadway Plaza (8526.07m) Jubilee Square (8342.17m) Junction 8 (7175.12m) Jurong East Primary School (600m) Fuhua Primary School (900m)"

density = embed_poi_density(text1)
print("POI Density:", density)

#### 2.2 Word embedding calculated in separate chunks due to slow runtime

In [103]:
import numpy as np

chunks = np.array_split(hdb_working_data, 100)

result_chunks = []

for i, chunk in enumerate(chunks[64:82]):
    print(f"Starting chunk: {i+1}")
    chunk['poi_vector'] = chunk.apply(lambda row: embed_poi_density(row['poi_mrt_prisch_mall']), axis=1)

    result_chunks.append(chunk)

    combined_df = pd.concat(result_chunks)
    combined_df.to_csv('../data/modified/hdb_working_data.csv', index=False)
    print(f"Finished chunk: {i+1}")

/Users/joesurf/Documents/NUS/BZA/Y3S2/BT4222/Housing-Price-Time-Series-Forecasting/env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Starting chunk: 1
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................